In [1]:
import numpy as np
import urllib
import librosa
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from microphone import record_audio
from IPython.display import Audio
from pathlib import Path
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion 
from scipy.ndimage.morphology import iterate_structure
import pickle
from collections import Counter
# make this a glboal variable - it is false unless audio_to_digital changes
Checking=False 

In [2]:
def audio_to_digital(*file_path_or_music):
    if(len(file_path_or_music)!= 0):
        if(".mp3" in file_path_or_music[0]):
            #analyze file input if it is there
            from pathlib import Path
            local_song_path = Path(file_path_or_music[0])
            frames, sample_rate = librosa.load(local_song_path, sr=44100, mono=True, duration=5) #Seconds
            audio_data = np.hstack([np.frombuffer(i, np.int16) for i in frames])

        else:
            #analyze file input if it is there
            with open(file_path_or_music[0], 'r') as R:
            # each sample is written to a single line in the text file
            # this reads them in as a single integer-valued numpy array
                audio_data = np.asarray([int(i) for i in R])
    else:
        #or else we are recording what they say
        from microphone import record_audio
        listen_time = 5  # seconds
        frames, sample_rate = record_audio(listen_time)
        # read in the recorded audio, saved as a numpy array of 16-bit integers
        audio_data = np.hstack([np.frombuffer(i, np.int16) for i in frames])
        Checking=True
    return audio_data

In [3]:
def digitalToSpecto(audio):
    S, freqs, times = mlab.specgram(audio, NFFT=4096, Fs=44100,
                                  window=mlab.window_hanning,
                                  noverlap=4096 // 2)
    return(S)

In [24]:
def spectrogram_to_peaks(arr):
    """ 
    Creates a boolean array showing peaks, given data from a spectrogram.
    
    Parameters:
        arr: The array produced by the spectrogram from digital_to_spectrogram with shape (N,M)
        
    Returns:
        peaks: A boolean array with shape (N,M). Peaks in the data are where peaks == True.
    """

    # Creating the histogram
    arr_flattened = np.log(arr.flatten() + 1e-20)
    
    N = arr_flattened.size # Number of elements in the array
    cnt, bin_edges = np.histogram(arr_flattened, bins=N//200, density=True)
    bin_width = np.diff(bin_edges) 
    
    ## print(np.sum(cnt*bin_width)) # check that summation = 1
    
    # Creating the cumulative distribution
    cumulative_distr = np.cumsum(cnt*bin_width)
    
    # Defining the cutoff
    frac_cut = 0.9
    bin_index_of_cutoff = np.searchsorted(cumulative_distr, frac_cut)
    
    # given the bin-index, we want the associated log-amplitude value for that bin
    cutoff_log_amplitude = bin_edges[bin_index_of_cutoff]
    
    # Defining the footprint
    fp = generate_binary_structure(rank=2,connectivity=2)
    fp = np.ones((20,20))
    
    peaks = ((arr > cutoff_log_amplitude) & (arr == maximum_filter(arr, footprint=fp)))
    freq, time = np.where(peaks)
    print(freq)
    print(len(freq))
    return peaks

In [25]:
def peaks_to_dic_newsongs(local_peaks):
    with open("songs.pkl", mode="rb") as opened_file:
        song_dic = pickle.load(opened_file)
    freq, time= np.where(local_peaks)
    song_name=input("Song Name= ")
    fanout = 15
    for i in range(len(local_peaks)-fanout):
        for j in range (1,fanout+1):
            finger=(freq[i], freq[i+j], time[i+j]-time[i])
            print(finger)
            song_dir=(song_name, time[i])
            song_dic.update([(finger, song_dir)])
    # open and save to a pkl file, mode = wb for binary storage w/ numpy array
    with open("songs.pkl", mode="wb") as opened_file:
        pickle.dump(song_dic, opened_file)
    print("Done")

In [26]:
def peaks_to_dic_findsong(local_peaks):
    with open("songs.pkl", mode="rb") as opened_file:
        song_dic2 = pickle.load(opened_file)
    freq, time= np.where(local_peaks)
    songList = []
    fingerList=[]
    for i in range(len(local_peaks)-15):
        j=1
        for j in range(16):
            finger=(freq[i], freq[i+j], time[i+j]-time[i])
            fingerList.append(finger)
    for i in range(len(fingerList)):
        if fingerList[i] in song_dic2:
            song_guess=song_dic2[finger]
            songList.append((song_guess[0], (song_guess[1]-finger[2])))
            Counter_songs= Counter(songList)
    song=Counter_songs.most_common(1)
    with open("songs.pkl", mode="wb") as opened_file:
        pickle.dump(song_dic2, opened_file)
    return song

In [28]:
def final_function(song, Checking=False):
    audio=audio_to_digital(song)
    arr=digitalToSpecto(audio)
    local_peaks=spectrogram_to_peaks(arr)
    if Checking==True:
        Checking=False
        peaks_to_dic_findsong(local_peaks)
    else:
        peaks_to_dic_newsongs(local_peaks)

In [29]:
final_function("songs/berhana - Janet.mp3")

[   0    0    0 ..., 2045 2045 2045]
1097
Song Name= Janet
(0, 0, 86)
(0, 0, 132)
(0, 0, 181)
(0, 3, -12)
(0, 3, 31)
(0, 3, 117)
(0, 3, 160)
(0, 8, 56)
(0, 8, 142)
(0, 18, 174)
(0, 20, 7)
(0, 20, 26)
(0, 21, 92)
(0, 21, 113)
(0, 31, 103)
(0, 0, 46)
(0, 0, 95)
(0, 3, -98)
(0, 3, -55)
(0, 3, 31)
(0, 3, 74)
(0, 8, -30)
(0, 8, 56)
(0, 18, 88)
(0, 20, -79)
(0, 20, -60)
(0, 21, 6)
(0, 21, 27)
(0, 31, 17)
(0, 32, 36)
(0, 0, 49)
(0, 3, -144)
(0, 3, -101)
(0, 3, -15)
(0, 3, 28)
(0, 8, -76)
(0, 8, 10)
(0, 18, 42)
(0, 20, -125)
(0, 20, -106)
(0, 21, -40)
(0, 21, -19)
(0, 31, -29)
(0, 32, -10)
(0, 36, 35)
(0, 3, -193)
(0, 3, -150)
(0, 3, -64)
(0, 3, -21)
(0, 8, -125)
(0, 8, -39)
(0, 18, -7)
(0, 20, -174)
(0, 20, -155)
(0, 21, -89)
(0, 21, -68)
(0, 31, -78)
(0, 32, -59)
(0, 36, -14)
(0, 39, -125)
(3, 3, 43)
(3, 3, 129)
(3, 3, 172)
(3, 8, 68)
(3, 8, 154)
(3, 18, 186)
(3, 20, 19)
(3, 20, 38)
(3, 21, 104)
(3, 21, 125)
(3, 31, 115)
(3, 32, 134)
(3, 36, 179)
(3, 39, 68)
(3, 41, 6)
(3, 3, 86)
(3, 3, 129)

(159, 167, -40)
(159, 169, -173)
(159, 172, 15)
(159, 173, -68)
(159, 174, -126)
(159, 177, -155)
(159, 179, -38)
(159, 182, 3)
(159, 184, -54)
(159, 185, -133)
(159, 185, -69)
(159, 185, -24)
(160, 163, 19)
(160, 166, -13)
(160, 167, 61)
(160, 169, -72)
(160, 172, 116)
(160, 173, 33)
(160, 174, -25)
(160, 177, -54)
(160, 179, 63)
(160, 182, 104)
(160, 184, 47)
(160, 185, -32)
(160, 185, 32)
(160, 185, 77)
(160, 188, 21)
(163, 166, -32)
(163, 167, 42)
(163, 169, -91)
(163, 172, 97)
(163, 173, 14)
(163, 174, -44)
(163, 177, -73)
(163, 179, 44)
(163, 182, 85)
(163, 184, 28)
(163, 185, -51)
(163, 185, 13)
(163, 185, 58)
(163, 188, 2)
(163, 194, -88)
(166, 167, 74)
(166, 169, -59)
(166, 172, 129)
(166, 173, 46)
(166, 174, -12)
(166, 177, -41)
(166, 179, 76)
(166, 182, 117)
(166, 184, 60)
(166, 185, -19)
(166, 185, 45)
(166, 185, 90)
(166, 188, 34)
(166, 194, -56)
(166, 196, 75)
(167, 169, -133)
(167, 172, 55)
(167, 173, -28)
(167, 174, -86)
(167, 177, -115)
(167, 179, 2)
(167, 182, 43)
(16

(387, 398, 34)
(387, 402, -51)
(387, 405, 79)
(387, 409, -5)
(387, 411, -22)
(387, 413, 21)
(387, 415, 66)
(387, 416, -64)
(387, 387, 87)
(387, 390, 34)
(387, 391, -97)
(387, 392, -33)
(387, 393, -87)
(387, 394, 70)
(387, 398, 23)
(387, 402, -62)
(387, 405, 68)
(387, 409, -16)
(387, 411, -33)
(387, 413, 10)
(387, 415, 55)
(387, 416, -75)
(387, 419, 75)
(387, 390, -53)
(387, 391, -184)
(387, 392, -120)
(387, 393, -174)
(387, 394, -17)
(387, 398, -64)
(387, 402, -149)
(387, 405, -19)
(387, 409, -103)
(387, 411, -120)
(387, 413, -77)
(387, 415, -32)
(387, 416, -162)
(387, 419, -12)
(387, 419, -2)
(390, 391, -131)
(390, 392, -67)
(390, 393, -121)
(390, 394, 36)
(390, 398, -11)
(390, 402, -96)
(390, 405, 34)
(390, 409, -50)
(390, 411, -67)
(390, 413, -24)
(390, 415, 21)
(390, 416, -109)
(390, 419, 41)
(390, 419, 51)
(390, 421, -138)
(391, 392, 64)
(391, 393, 10)
(391, 394, 167)
(391, 398, 120)
(391, 402, 35)
(391, 405, 165)
(391, 409, 81)
(391, 411, 64)
(391, 413, 107)
(391, 415, 152)
(391,

(631, 653, 194)
(631, 654, 46)
(631, 656, 141)
(631, 657, 107)
(631, 631, 25)
(631, 632, 173)
(631, 633, 151)
(631, 644, 130)
(631, 645, 161)
(631, 646, 34)
(631, 647, 11)
(631, 647, 70)
(631, 650, 96)
(631, 651, -1)
(631, 653, 172)
(631, 654, 24)
(631, 656, 119)
(631, 657, 85)
(631, 658, 152)
(631, 632, 148)
(631, 633, 126)
(631, 644, 105)
(631, 645, 136)
(631, 646, 9)
(631, 647, -14)
(631, 647, 45)
(631, 650, 71)
(631, 651, -26)
(631, 653, 147)
(631, 654, -1)
(631, 656, 94)
(631, 657, 60)
(631, 658, 127)
(631, 660, -15)
(632, 633, -22)
(632, 644, -43)
(632, 645, -12)
(632, 646, -139)
(632, 647, -162)
(632, 647, -103)
(632, 650, -77)
(632, 651, -174)
(632, 653, -1)
(632, 654, -149)
(632, 656, -54)
(632, 657, -88)
(632, 658, -21)
(632, 660, -163)
(632, 661, -184)
(633, 644, -21)
(633, 645, 10)
(633, 646, -117)
(633, 647, -140)
(633, 647, -81)
(633, 650, -55)
(633, 651, -152)
(633, 653, 21)
(633, 654, -127)
(633, 656, -32)
(633, 657, -66)
(633, 658, 1)
(633, 660, -141)
(633, 661, -162)


(852, 868, -147)
(852, 868, 42)
(852, 872, -32)
(852, 873, -87)
(852, 874, -20)
(852, 877, -43)
(855, 857, -18)
(855, 858, 32)
(855, 859, 76)
(855, 862, 43)
(855, 862, 129)
(855, 862, 140)
(855, 866, 11)
(855, 866, 156)
(855, 868, -17)
(855, 868, 172)
(855, 872, 98)
(855, 873, 43)
(855, 874, 110)
(855, 877, 87)
(855, 878, -21)
(857, 858, 50)
(857, 859, 94)
(857, 862, 61)
(857, 862, 147)
(857, 862, 158)
(857, 866, 29)
(857, 866, 174)
(857, 868, 1)
(857, 868, 190)
(857, 872, 116)
(857, 873, 61)
(857, 874, 128)
(857, 877, 105)
(857, 878, -3)
(857, 879, 45)
(858, 859, 44)
(858, 862, 11)
(858, 862, 97)
(858, 862, 108)
(858, 866, -21)
(858, 866, 124)
(858, 868, -49)
(858, 868, 140)
(858, 872, 66)
(858, 873, 11)
(858, 874, 78)
(858, 877, 55)
(858, 878, -53)
(858, 879, -5)
(858, 879, 23)
(859, 862, -33)
(859, 862, 53)
(859, 862, 64)
(859, 866, -65)
(859, 866, 80)
(859, 868, -93)
(859, 868, 96)
(859, 872, 22)
(859, 873, -33)
(859, 874, 34)
(859, 877, 11)
(859, 878, -97)
(859, 879, -49)
(859, 87

...

(1234, 1253, -5)
(1234, 1254, -42)
(1234, 1257, 27)
(1234, 1259, -69)
(1234, 1259, 17)
(1234, 1263, -91)
(1236, 1237, -111)
(1236, 1238, -159)
(1236, 1240, -25)
(1236, 1242, 15)
(1236, 1244, -43)
(1236, 1247, -11)
(1236, 1249, 0)
(1236, 1252, -23)
(1236, 1253, -75)
(1236, 1254, -112)
(1236, 1257, -43)
(1236, 1259, -139)
(1236, 1259, -53)
(1236, 1263, -161)
(1236, 1266, -10)
(1237, 1238, -48)
(1237, 1240, 86)
(1237, 1242, 126)
(1237, 1244, 68)
(1237, 1247, 100)
(1237, 1249, 111)
(1237, 1252, 88)
(1237, 1253, 36)
(1237, 1254, -1)
(1237, 1257, 68)
(1237, 1259, -28)
(1237, 1259, 58)
(1237, 1263, -50)
(1237, 1266, 101)
(1237, 1269, 68)
(1238, 1240, 134)
(1238, 1242, 174)
(1238, 1244, 116)
(1238, 1247, 148)
(1238, 1249, 159)
(1238, 1252, 136)
(1238, 1253, 84)
(1238, 1254, 47)
(1238, 1257, 116)
(1238, 1259, 20)
(1238, 1259, 106)
(1238, 1263, -2)
(1238, 1266, 149)
(1238, 1269, 116)
(1238, 1274, 0)
(1240, 1242, 40)
(1240, 1244, -18)
(1240, 1247, 14)
(1240, 1249, 25)
(1240, 1252, 2)
(1240, 1253,

(1429, 1432, 120)
(1429, 1433, 11)
(1429, 1437, 139)
(1429, 1437, 171)
(1429, 1438, 107)
(1429, 1443, 30)
(1429, 1443, 97)
(1429, 1444, 75)
(1429, 1447, -18)
(1429, 1451, 124)
(1429, 1452, 41)
(1430, 1431, 29)
(1430, 1432, -19)
(1430, 1432, 43)
(1430, 1432, 77)
(1430, 1433, -32)
(1430, 1437, 96)
(1430, 1437, 128)
(1430, 1438, 64)
(1430, 1443, -13)
(1430, 1443, 54)
(1430, 1444, 32)
(1430, 1447, -61)
(1430, 1451, 81)
(1430, 1452, -2)
(1430, 1455, 109)
(1431, 1432, -48)
(1431, 1432, 14)
(1431, 1432, 48)
(1431, 1433, -61)
(1431, 1437, 67)
(1431, 1437, 99)
(1431, 1438, 35)
(1431, 1443, -42)
(1431, 1443, 25)
(1431, 1444, 3)
(1431, 1447, -90)
(1431, 1451, 52)
(1431, 1452, -31)
(1431, 1455, 80)
(1431, 1458, 99)
(1432, 1432, 62)
(1432, 1432, 96)
(1432, 1433, -13)
(1432, 1437, 115)
(1432, 1437, 147)
(1432, 1438, 83)
(1432, 1443, 6)
(1432, 1443, 73)
(1432, 1444, 51)
(1432, 1447, -42)
(1432, 1451, 100)
(1432, 1452, 17)
(1432, 1455, 128)
(1432, 1458, 147)
(1432, 1462, -13)
(1432, 1432, 34)
(1432, 1

(1640, 1661, -44)
(1640, 1662, -119)
(1640, 1662, -75)
(1640, 1662, -1)
(1645, 1646, -25)
(1645, 1647, -101)
(1645, 1650, -45)
(1645, 1651, -173)
(1645, 1651, -154)
(1645, 1654, -129)
(1645, 1655, -69)
(1645, 1656, -14)
(1645, 1657, -144)
(1645, 1661, -111)
(1645, 1661, -25)
(1645, 1662, -100)
(1645, 1662, -56)
(1645, 1662, 18)
(1645, 1663, -82)
(1646, 1647, -76)
(1646, 1650, -20)
(1646, 1651, -148)
(1646, 1651, -129)
(1646, 1654, -104)
(1646, 1655, -44)
(1646, 1656, 11)
(1646, 1657, -119)
(1646, 1661, -86)
(1646, 1661, 0)
(1646, 1662, -75)
(1646, 1662, -31)
(1646, 1662, 43)
(1646, 1663, -57)
(1646, 1670, 28)
(1647, 1650, 56)
(1647, 1651, -72)
(1647, 1651, -53)
(1647, 1654, -28)
(1647, 1655, 32)
(1647, 1656, 87)
(1647, 1657, -43)
(1647, 1661, -10)
(1647, 1661, 76)
(1647, 1662, 1)
(1647, 1662, 45)
(1647, 1662, 119)
(1647, 1663, 19)
(1647, 1670, 104)
(1647, 1673, 44)
(1650, 1651, -128)
(1650, 1651, -109)
(1650, 1654, -84)
(1650, 1655, -24)
(1650, 1656, 31)
(1650, 1657, -99)
(1650, 1661, 

(1861, 1891, 3)
(1861, 1892, 60)
(1861, 1893, 17)
(1861, 1894, 71)
(1862, 1862, 86)
(1862, 1863, 27)
(1862, 1866, 72)
(1862, 1868, 13)
(1862, 1870, -99)
(1862, 1879, 55)
(1862, 1880, -14)
(1862, 1882, -85)
(1862, 1882, -45)
(1862, 1885, 35)
(1862, 1891, -57)
(1862, 1892, 0)
(1862, 1893, -43)
(1862, 1894, 11)
(1862, 1896, 75)
(1862, 1863, -59)
(1862, 1866, -14)
(1862, 1868, -73)
(1862, 1870, -185)
(1862, 1879, -31)
(1862, 1880, -100)
(1862, 1882, -171)
(1862, 1882, -131)
(1862, 1885, -51)
(1862, 1891, -143)
(1862, 1892, -86)
(1862, 1893, -129)
(1862, 1894, -75)
(1862, 1896, -11)
(1862, 1898, -171)
(1863, 1866, 45)
(1863, 1868, -14)
(1863, 1870, -126)
(1863, 1879, 28)
(1863, 1880, -41)
(1863, 1882, -112)
(1863, 1882, -72)
(1863, 1885, 8)
(1863, 1891, -84)
(1863, 1892, -27)
(1863, 1893, -70)
(1863, 1894, -16)
(1863, 1896, 48)
(1863, 1898, -112)
(1863, 1900, -128)
(1866, 1868, -59)
(1866, 1870, -171)
(1866, 1879, -17)
(1866, 1880, -86)
(1866, 1882, -157)
(1866, 1882, -117)
(1866, 1885, -37

IndexError: index 1097 is out of bounds for axis 0 with size 1097